In [251]:
import random
# Redo hand if three people enter the flop.

# Opponents = 6 max and 100 man tournament; 10 paid:   Just assume 9 max is 6 max table
    # 30, 20, 12, 9.5, 8, 6, 5, 4, 3, 2.5 is the payout

# ICM calculated by % of chip pool you have looking for first place chances.  
    # Then the ICM is calculated by assuming you do not win first; do the same with the remaining players.  
    # Assuming other combinations got first.
    
# ICM isn't calculated in small blind sizes until there are 12 opponents left
    # Therefore this program will ignore ICM completely.
    
    
################# Strategy #################
def get_randomized_strategies():
    strategies = []
    for i in range(6):
        strategy = get_randomized_strategy()
        strategies.append(strategy)
    return strategies

def get_randomized_strategy():
    strategy = []
    choices = [3, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20]
    length_of_strategy = 40
    for i in range(length_of_strategy):
        strategy.append(random.choices(choices)[0])
    return strategy




################# Monte Carlo #################
def get_average_profit_from_playing_hands(hands):
    profit = 0
    for i in range(hands):        
        profit_from_playing_hand = get_profit_from_playing_hand()
        profit += profit_from_playing_hand
    avg_profit = profit / hands
    return avg_profit



################# Playing a Hand #################
def get_profit_from_playing_hand(strategies):
    hands_remaining, board, range1, range2 = play_preflop()
    hands, deck, my_winnings_given_postflop, board = play_flop(strategies, my_position_id, opponent_position_id, hands, deck)
    if my_winnings_given_postflop != None:
        return my_winnings_given_postflop    
    hands, deck, my_winnings_given_postflop, board = play_turn(strategies, my_position_id, opponent_position_id, hands, deck)
    if my_winnings_given_postflop != None:
        return my_winnings_given_postflop    
    my_winnings_given_postflop = play_river(strategies, my_position_id, opponent_position_id, hands, deck)
    return my_winnings_given_postflop

def play_preflop():
    players_remaining = 6
    raises = 0
    while players_remaining != 2:
        players_remaining = 6
        raises = 0        
        hands, deck = get_randomized_9_hands()
        raiser_position = None
        opponent_number = 0
        # LJ Turn
        action1 = get_players_first_and_second_and_third_preflop_action(1, raises, raiser_position, hands[0])
        if "r" in action1:
            raises += 1
            raiser_position = 1
        elif action1 == "f":
            players_remaining -= 1
        # HJ Turn
        action2 = get_players_first_and_second_and_third_preflop_action(2, raises, raiser_position, hands[1])
        if "r" in action2:
            raises += 1
            raiser_position = 2
        elif action2 == "f":
            players_remaining -= 1            
        # CO Turn
        action3 = get_players_first_and_second_and_third_preflop_action(3, raises, raiser_position, hands[2])
        if "r" in action3:
            raises += 1
            raiser_position = 3
        elif action3 == "f":
            players_remaining -= 1            
        # BN Turn
        action4 = get_players_first_and_second_and_third_preflop_action(4, raises, raiser_position, hands[3])
        if "r" in action4:
            raises += 1
            raiser_position = 4
        elif action4 == "f":
            players_remaining -= 1            
        # BN Turn
        action5 = get_players_first_and_second_and_third_preflop_action(5, raises, raiser_position, hands[4])
        if "r" in action5:
            raises += 1
            raiser_position = 5
        elif action5 == "f":
            players_remaining -= 1            
        # BN Turn
        action6 = get_players_first_and_second_and_third_preflop_action(6, raises, raiser_position, hands[5])
        if "r" in action6:
            raises += 1
            raiser_position = 6
        elif action6 == "f":
            players_remaining -= 1  
        preflop_actions = [action1, action2, action3, action4, action5, action6]
        unique_players_with_a_raise = sum([1 for x in preflop_actions if 'r' in x])
        temp = [x for x in preflop_actions if x != 'f']
        if len(temp) != 2:
            players_remaining = 1
            continue
        if unique_players_with_a_raise == 3 and temp[0] == 'rr':
            preflop_actions = [x[0:3] for x in preflop_actions]
        elif unique_players_with_a_raise == 2:
            preflop_actions = [x[0:2] for x in preflop_actions]
        elif unique_players_with_a_raise == 1:
            preflop_actions = [x[0] for x in preflop_actions]        
        if len(temp) == 2 and 'f' in temp[0] and temp[1].count('r') >= temp[0].count('r'):
            players_remaining = 1
            continue
    board = deck[0:5]
    
    hands_remaining = []
    for i, x in enumerate([x != 'f' for x in preflop_actions]):
        if x:
            hands_remaining.append(hands[i])

    range1 = get_range_player_1(preflop_actions)
    range2 = get_range_player_2(preflop_actions)
    return hands_remaining, board, range1, range2
    
def get_players_first_and_second_and_third_preflop_action(seat_at_the_table, raises, raiser_position, hand):
    first_and_second_action = get_players_first_and_second_preflop_action(seat_at_the_table, raises, raiser_position, hand)
    if first_and_second_action == 'rr':
        if hand in [[14,14], [13,13]]:
            return 'rrr'
        else:
            return 'rrf'
    else:
        return first_and_second_action
    
def get_players_first_and_second_preflop_action(seat_at_the_table, raises, raiser_position, hand):
    first_action = get_players_first_preflop_action(seat_at_the_table, raises, raiser_position, hand)
    if first_action != 'r':
        return first_action
    
    h = hand[2]
    suited = hand[3]
    paired = len(set(h)) == 1
    if suited:
        if (h == [14,13]) or (h in [[14,5], [14,4]]):
            return 'rr'
        elif h[1] >= 10:
            return 'rc'                
        else:
            return 'rf'
    elif paired:
        if h[0] >= 12:
            return 'rr'                
        elif h[0] >= 7:
            return 'rc'
        else:
            return 'rf'
    else:
        if h == [14,13]:
            return 'rr'
        elif (h == [14,12]):                    
            return 'rc'                
        else:
            return 'rf'  
    
    
def get_players_first_preflop_action(seat_at_the_table, raises, raiser_position, hand):
    h = hand[2]
    suited = hand[3]
    paired = len(set(h)) == 1  
    if raises == 0:
        if seat_at_the_table == 1:
            if suited:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f'
            elif paired:
                if h[0] >= 5:
                    return 'r'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 2:
            if suited:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 11:
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 3:
            if suited:
                if (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 10:
                    return 'r'
                else:
                    return 'f'   
        elif seat_at_the_table == 4:
            if suited:
                if (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f' 
        elif seat_at_the_table == 5:
            if suited:
                if (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 6:
            if suited:
                if (h[1] >= 9):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                if h[0] >= 9:
                    return 'r'
                else:
                    return 'f'
            else:
                if (h[1] >= 12):
                    return 'r'
                else:
                    return 'f'
    elif raises == 1:
        if seat_at_the_table == 6 and raiser_position == 1:
            if suited:
                if h in [[14,13], [14,12]]:
                    return 'r'                
                elif h[1] >= 9:
                    return 'c'
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 11:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 2:
            if suited:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'
                elif (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 10:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 3:
            if suited:
                if h in [[14,13], [14,12], [14,11], [13,12], [12,11], [11,10]]:
                    return 'r'
                elif (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 11:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 9:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 4:
            if suited:
                if h[1] >= 10:
                    return 'r'
                elif (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 10:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [13,12]]:
                    return 'r'
                elif (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'c'                
                else:
                    return 'f'  
        elif seat_at_the_table == 6 and raiser_position == 5:
            if suited:
                if h[1] >= 9:
                    return 'r'
                elif (h[1] >= 4) or (h[0] == 14) or (h[0]-h[1] <= 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 9:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [13,12]]:
                    return 'r'
                elif (h[1] >= 8):
                    return 'c'                
                else:
                    return 'f' 
        elif seat_at_the_table == 5 and raiser_position == 4:
            if suited:
                if h[1] >= 9:
                    return 'r'                            
                else:
                    return 'f'
            elif paired:
                if h[0] >= 5:
                    return 'r'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'            
                else:
                    return 'f'
        else:
            if suited:
                if (h == [14,13]) or (h in [[14,5], [14,4]]):
                    return 'r'
                elif h[1] >= 10:
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 7:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif (h == [14,12]):                    
                    return 'c'                
                else:
                    return 'f'            
    else:                
        if h in [[14,14], [13,13]]:
            return 'r'
        else:
            return 'f'

    
################# Shuffle and Deal #################
def get_randomized_9_hands():
    deck = shuffle_a_deck_of_cards()
    hands, deck = deal_two_cards_to_each(deck)
    return hands, deck

def shuffle_a_deck_of_cards():
    deck = []
    for i in range(2, 15):
        for j in range(1,5):
            deck.append([i, j])
    random.shuffle(deck)
    return deck
       
def deal_two_cards_to_each(deck):
    hands = [deck[0:2] + [sorted([deck[0][0], deck[1][0]], reverse=True)] + [deck[0][1] == deck[1][1]], 
             deck[2:4] + [sorted([deck[2][0], deck[3][0]], reverse=True)] + [deck[2][1] == deck[3][1]], 
             deck[4:6] + [sorted([deck[4][0], deck[5][0]], reverse=True)] + [deck[4][1] == deck[5][1]], 
             deck[6:8] + [sorted([deck[6][0], deck[7][0]], reverse=True)] + [deck[6][1] == deck[7][1]], 
             deck[8:10] + [sorted([deck[8][0], deck[9][0]], reverse=True)] + [deck[8][1] == deck[9][1]], 
             deck[10:12] + [sorted([deck[10][0], deck[11][0]], reverse=True)] + [deck[10][1] == deck[11][1]]]
    deck = deck[12:]
    return hands, deck

################# Flop #################
    
def play_flop(strategies, my_position_id, opponent_position_id, hands, deck):
    flop, deck = get_randomized_new_cards(deck=deck, number_of_cards=3)
    return hands, deck, my_winnings_given_postflop, board

def play_turn(strategies, my_position_id, opponent_position_id, hands, deck):
    turn, deck = get_randomized_new_cards(deck=deck, number_of_cards=1)
    return hands, deck, my_winnings_given_postflop, board

def play_river(strategies, my_position_id, opponent_position_id, hands, deck):
    river, deck = get_randomized_new_cards(deck=deck, number_of_cards=1)
    return my_winnings_given_postflop
    


def get_randomized_new_cards():
    pass




def get_range_player_1(actions):
    if sum(['r' in x for x in actions]) > 1:
        return 12
    elif 'r' in actions[0] and sum(['r' in x for x in actions[1:]]) == 0:
        return 1
    elif 'r' in actions[1] and sum(['r' in x for x in (actions[0:1] + actions[2:])]) == 0:
        return 2
    elif 'r' in actions[2] and sum(['r' in x for x in (actions[0:2] + actions[3:])]) == 0:
        return 3
    elif 'r' in actions[3] and sum(['r' in x for x in (actions[0:3] + actions[4:])]) == 0:
        return 4
    elif 'r' in actions[4] and sum(['r' in x for x in (actions[0:4] + actions[5:])]) == 0:
        return 5
    elif sum(['c' == x or 'rc' == x for x in actions]) == 1:
        return 11
    else:
        return 12
    
def get_range_player_2(actions):
    if sum(['r' in x for x in actions]) > 1:
        return 12    
    elif 'c' == actions[5] and 'r' == actions[0][0]:
        return 6
    elif 'c' == actions[5] and 'r' == actions[1][0]:
        return 7
    elif 'c' == actions[5] and 'r' == actions[2][0]:
        return 8
    elif 'c' == actions[5] and 'r' == actions[3][0]:
        return 9
    elif 'c' == actions[5] and 'r' == actions[4][0]:
        return 10
    elif sum(['c' == x or 'rc' == x for x in actions]) == 1:
        return 11  
    else:
        return 12    

    
    
    





In [252]:
# For show: (Invalid means there is no strategy, you have to solve it yourself.)
{1: 'lj rfi', 2: 'hj rfi', 3: 'co rfi', 4: 'bn rfi', 5: 'sb rfi',
 6: 'bb vs lj call', 7: 'bb vs hj call', 8: 'bb vs co call',
 9: 'bb vs bn call', 10: 'bb vs sb call', 11: 'other calls',
 12: 'invalid'}

# Invested based on player
{1: 2.5, 2: 2.5, 3: 2.5, 4: 2.5, 5: 2.5,
 6: 1.5, 7: 1.5, 8: 1.5, 9: 1.5, 10: 1.5, 11: 2.5, 12: 0}

# Pot size based on player 2
{6: 1.5, 7: 1.5, 8: 1.5, 9: 1.5, 10: 1.5, 11: 2.5, 12: 0}

strategies = get_randomized_strategies()
# avg_profit = get_average_profit_from_playing_hands(hands=100)
hands_remaining, board, range1, range2 = play_preflop()
hands_remaining, board, range1, range2


([[[9, 4], [14, 3], [14, 9], False], [[13, 1], [9, 2], [13, 9], False]],
 [[14, 1], [14, 2], [2, 2], [14, 4], [6, 4]],
 4,
 9)

In [253]:
hands_remaining, board[0:3], range1, range2

([[[9, 4], [14, 3], [14, 9], False], [[13, 1], [9, 2], [13, 9], False]],
 [[14, 1], [14, 2], [2, 2]],
 4,
 9)

(defaultdict(int, {4: 1855, 3: 1966, 1: 1315, 12: 1362, 2: 1427, 5: 2075}),
 defaultdict(int,
             {9: 1855, 8: 1464, 6: 487, 12: 1362, 7: 759, 11: 1998, 10: 2075}))

In [ ]:
from collections import defaultdict
temp1 = defaultdict(int)
temp2 = defaultdict(int)
for i in range(10000):
    hands_remaining, board, range1, range2 = play_preflop()
    temp1[range1] += 1
    temp2[range2] += 1
temp1, temp2